In [1]:
import requests
import pandas as pd
import json


In [2]:
#pega id's de todos os cavalos

pega_dic = {'market_id':0, 'token_id':0, 'price':0, 'reward':0,'total_races':0,
                'win':0, 'lose':0, 'speed':0, 'strength':0, 'wind':0,
                    'water':0, 'fire':0, 'lighting':0, 'energy':0,
                        '1':0, '2':0, '3':0, '4':0, 
                            '5':0, '6':0, '7':0, '8':0, 
                                '9':0, '10':0, '11':0, '12':0}


pega_excel = pd.DataFrame()

In [3]:
def code(pega_dic, pega_excel, page_number):


    if(page_number == 0):
        market_url = 'https://api-apollo.pegaxy.io/v1/game-api/market/pegaslisting/'
    else:
        market_url = 'https://api-apollo.pegaxy.io/v1/game-api/market/pegaslisting/'+ str(page_number)
    
    try:
        req_market = requests.get(market_url)
        req_market = req_market.json()
    except:
        return pega_dic


    for pega_market in range(0,12):

        pega_dic['market_id'] = req_market['market'][pega_market]['id']
        pega_dic['token_id'] = req_market['market'][pega_market]['pega']['id']
        pega_dic['price'] = req_market['market'][pega_market]['price']

        pegadata_url = 'https://api-apollo.pegaxy.io/v1/game-api/pega/'+str(pega_dic['token_id'])
        
        try:
            req_pegadata = requests.get(pegadata_url)
            req_pegadata = req_pegadata.json()
            pega_api = req_pegadata['pega']
        except:
            continue


        pega_dic['total_races'] = pega_api['total_races']
        pega_dic['win'] = pega_api['win']
        pega_dic['lose'] = pega_api['lose']
        pega_dic['speed'] = pega_api['speed']
        pega_dic['strength'] = pega_api['strength']
        pega_dic['wind'] = pega_api['wind']
        pega_dic['water'] = pega_api['water']
        pega_dic['fire'] = pega_api['fire']
        pega_dic['lighting'] = pega_api['lighting']
        pega_dic['energy'] = pega_api['energy']
        pega_dic['reward'] = 0
        
        try:
            req_race = requests.get('https://api-apollo.pegaxy.io/v1/game-api/race/history/pega/'+str(pega_dic['token_id']))
            req_race = req_race.json()
            req_race['data']
        except:
            continue

    
       
        position_count = []

        for pega_race in req_race['data']:

            #pode implementar mais coisas relacionadas a corrida aqui
            pega_dic['reward'] = pega_dic['reward'] + pega_race['reward'] 
            
            #sistema das corridas
            position_count.append(pega_race['position'])
            

        for i in range(1,13):
            pega_dic[str(i)] = position_count.count(i)

        
        pega_excel = pega_excel.append(pega_dic, ignore_index=True)

    return pega_excel

In [4]:
import concurrent.futures
import collections
import multiprocessing

pega_df = pd.DataFrame()

with concurrent.futures.ProcessPoolExecutor(64) as executor:

    run = [executor.submit(code, pega_dic, pega_excel, page_number) for page_number in range(1,136)]

    for f in concurrent.futures.as_completed(run):
        dic = f.result()
        pega_excel = pega_excel.append(dic, ignore_index=True)


In [5]:
pega_excel

,1,10,11,12,2,3,4,5,6,7,...,market_id,price,reward,speed,strength,token_id,total_races,water,win,wind
0,8.0,3.0,9.0,1.0,9.0,3.0,3.0,7.0,8.0,13.0,...,270662.0,400000000,1314.0,0.0,7.0,281659.0,87.0,6.0,20.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,270655.0,490000000,0.0,4.0,4.0,311865.0,0.0,6.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,270653.0,370000000,0.0,8.0,1.0,286879.0,0.0,4.0,0.0,3.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,270652.0,359000000,0.0,1.0,8.0,315450.0,0.0,8.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,270651.0,358000000,0.0,3.0,4.0,315506.0,0.0,3.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,191254.0,1500000000,0.0,7.0,1.0,191627.0,0.0,4.0,0.0,6.0
757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,191202.0,250000000000,0.0,3.0,6.0,198346.0,0.0,5.0,0.0,2.0
758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189604.0,3999000000,0.0,4.0,7.0,106201.0,0.0,1.0,0.0,6.0
759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189562.0,12999000000,0.0,8.0,8.0,40160.0,0.0,6.0,0.0,8.0


In [6]:
pega_excel = pega_excel.reindex(columns=['market_id', 'token_id' , 'price' , 'reward' ,'total_races' ,
                'win' , 'lose' , 'speed' , 'strength' , 'wind' ,
                    'water' , 'fire' , 'lighting' , 'energy' ,
                        '1' , '2' , '3' , '4' , 
                            '5' , '6' , '7' , '8' , 
                                '9' , '10' , '11' , '12' ])

                        
    
from datetime import datetime as dt

key = '{}{}{}'.format(dt.now().hour,dt.now().minute, dt.now().second)

pega_excel.to_excel('PEGA_analisys{}.xls'.format(key))